# Pan Line 3 Coil Change Analysis
Dataset May 05, 2021 to December 02, 2025

1. Python configuration and requirements
   1. Create a Python virtual environment: `python3 -m venv ~/Python/venv/jupyter`
   2. Activate environment: `source ~/Python/venv/jupyter/bin/activate`
   3. Required packages are listed in *requirements.txt* file 
   4. Install required packages: `pip install -r requirements.txt`
2. Import Pan Line data and process: *data/Line 3 CoilUsage 2021_05_12-2025_12_02.csv*
   1. Set date data type for dates
   2. Extract year and day of year for production and load dates
   3.  Extract stile gauge from Haas Smart Part Number
   4.  Compare coil ID to previous to determine if the coil has changed
3.  Create a data field with unique coils per day and coil changes per day

In [ ]:
import pandas as pd
from IPython.display import Markdown as md

df = pd.read_csv('data/Line 3 CoilUsage 2021_05_12-2025_12_02.csv')

# # Set date type for date columns
df['DateTimeStamp'] = pd.to_datetime(df['DateTimeStamp'])
df['LoadDate'] = pd.to_datetime(df['LoadDate'])

# # Extract day of year and year from date columns
df['DayOfYear'] = df['DateTimeStamp'].dt.dayofyear
df['LoadDayOfYear'] = df['LoadDate'].dt.dayofyear
df['LoadYear'] = df['LoadDate'].dt.year
df['Year'] = df['DateTimeStamp'].dt.year
df['Month'] = df['DateTimeStamp'].dt.month

# # Make a column for the stile gauge
df['StileGauge'] = df['SectionNumber'].str[12:13]

# # Make a copy of the CoilId column and shift down by one
# # then compare to see if the coil changed
df['IsCoilChange'] = df['HaasCoilNum'].shift(1) != df['HaasCoilNum']

# # Count the number of unique coils used per day of each year
unique_coils_per_day_year = df.groupby(by=['Year', 'DayOfYear'], group_keys=True)['HaasCoilNum'].nunique()

# Count the number of coils changed per day of each year
coils_changed_per_day_year = df.groupby(by=['Year', 'DayOfYear'], group_keys=True)['IsCoilChange'].sum()

# Merge the two series into the DataFrame
coil_changes_year = pd.merge(unique_coils_per_day_year, coils_changed_per_day_year, on=['Year', 'DayOfYear'], suffixes=('_UniqueCoils', '_CoilsChanged'))
coil_changes_year = coil_changes_year.rename(columns={'HaasCoilNum': 'Unique Coils', 'IsCoilChange': 'Coils Changed'})

# Generate summary markdown
data_preview = f"""
### Data Preview
There are {len(df)} pan sections across {df['Year'].nunique()} years. Here's a table of the first five rows of the DataFrame:
{df.head(5).to_markdown(index=False)}
"""

display(md(data_preview))


## Data Preview
There are 1060346 pan sections across 5 years. Here's a table of the first five rows of the DataFrame:
| DateTimeStamp              | SectionNumber                  | HaasCoilNum                 | LoadDate            |   DayOfYear |   LoadDayOfYear |   LoadYear |   Year |   Month |   StileGauge | IsCoilChange   |
|:---------------------------|:-------------------------------|:----------------------------|:--------------------|------------:|----------------:|-----------:|-------:|--------:|-------------:|:---------------|
| 2021-05-12 00:00:08.607000 | T21PW-DBSS-N5-1000NS-NNNNN-NXN | PW-SC-DS-17-27.200-00044987 | 2021-05-13 00:00:00 |         132 |             133 |       2021 |   2021 |       5 |            5 | True           |
| 2021-05-12 00:00:35.673000 | B21PW-DBSS-N5-1000NS-NNNNN-NXL | PW-SC-DS-17-27.200-00044987 | 2021-05-13 00:00:00 |         132 |             133 |       2021 |   2021 |       5 |            5 | False          |
| 2021-05-12 00:01:03.290000 | I21PW-DBSS-N5-1000NS-NNNNN-NXN | PW-SC-DS-17-27.200-00044987 | 2021-05-13 00:00:00 |         132 |             133 |       2021 |   2021 |       5 |            5 | False          |
| 2021-05-12 00:01:31.227000 | I21PW-DBSS-N5-1000NS-NNNNN-NXN | PW-SC-DS-17-27.200-00044987 | 2021-05-13 00:00:00 |         132 |             133 |       2021 |   2021 |       5 |            5 | False          |
| 2021-05-12 00:01:58.243000 | T21PW-DBSS-N5-1000NS-NNNNN-NXN | PW-SC-DS-17-27.200-00044987 | 2021-05-13 00:00:00 |         132 |             133 |       2021 |   2021 |       5 |            5 | False          |


In [ ]:
# Create a dataframe showing the total coil changes per month for each year
coil_changes_month = df.groupby(by=['Year', 'Month'], group_keys=True)['IsCoilChange'].sum().unstack(level=0)
coil_changes_month.index = coil_changes_month.index.map(lambda x: pd.to_datetime(str(x), format='%m').strftime('%b'))  

display(md(coil_changes_month.to_markdown()))

| Month   |   2021 |   2022 |   2023 |   2024 |   2025 |
|:--------|-------:|-------:|-------:|-------:|-------:|
| Jan     |    nan |    615 |    752 |    751 |    733 |
| Feb     |    nan |    608 |    671 |    590 |    530 |
| Mar     |    nan |    732 |    808 |    557 |    650 |
| Apr     |    nan |    601 |    740 |    612 |    598 |
| May     |    427 |    755 |    889 |    818 |    716 |
| Jun     |    493 |    758 |    887 |    632 |    749 |
| Jul     |    440 |    694 |    757 |    648 |    724 |
| Aug     |    636 |    935 |    832 |    605 |    667 |
| Sep     |    500 |    798 |    750 |    612 |    702 |
| Oct     |    535 |    844 |    857 |    720 |    770 |
| Nov     |    545 |    754 |    626 |    642 |    690 |
| Dec     |    462 |    600 |    494 |    535 |     79 |

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

print(coil_changes_month.head())

# graph comparing years of coil changes per month skipping NaN values



